In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import tensorflow as tf
import glob
import numpy as np
from PIL import Image
from sklearn.metrics import pairwise_distances, roc_auc_score
from matplotlib import pyplot as plt
%matplotlib inline
from tqdm import tqdm
import sys
sys.path.append("../")
from inception_resnet_v1 import inference
import pandas as pd
import seaborn as sns

In [2]:
layer_to_target = "Conv2d_4b_3x3"

In [3]:
def prewhiten(x):
    """
    A helper function to whiten an image, or a batch of images.
    Args:
        x: An image or batch of images.
    """
    if x.ndim == 4:
        axis = (1, 2, 3)
        size = x[0].size
    elif x.ndim == 3:
        axis = (0, 1, 2)
        size = x.size
    else:
        print(x.ndim)
        raise ValueError('Dimension should be 3 or 4')

    mean = np.mean(x, axis=axis, keepdims=True)
    std = np.std(x, axis=axis, keepdims=True)
#     std_adj = np.maximum(std, 1.0/np.sqrt(size))
    y = (x - mean) / std
    return y

def l2_normalize(x, axis=-1, epsilon=1e-10):
    """
    Normalizes an embedding to have unit length in the l2 metric.
    Args:
        x: A batch of numpy embeddings
    """
    output = x / np.sqrt(np.maximum(np.sum(np.square(x),
                                           axis=axis,
                                           keepdims=True),
                                    epsilon))
    return output

In [4]:
class PersonImages:
    def __init__(self, person_name):
        self.clean_folder = ""
        self.clean_images = []
        self.adversarial_images = []
        self.orig_mean = None
        self.orig_std = None
        self.orig_paths = []
        self.person_name = person_name
    
    def _load_one_facenet(self, path, resize_size=None, prewhiten_img=True):
        img = Image.open(path).convert("RGB")
    
            
        if resize_size:
            img = img.resize((resize_size, resize_size))
            
        img = (np.array(img)).astype(np.float32)
        
        if prewhiten_img:
            img = prewhiten(img)
            
        return img
        
    def _load_folder_for_facenet(self, folder, resize_size=None):
        paths_list = glob.glob(os.path.join(folder, "*"))
        final_imgs = []
        for img_path in paths_list:
            final_imgs.append(
                self._load_one_facenet(
                    img_path, prewhiten_img=False, resize_size=resize_size))
        
        final_imgs = np.array(final_imgs)
        mean, std = np.mean(final_imgs), np.std(final_imgs)
        final_imgs = prewhiten(np.array(final_imgs))
        return final_imgs, mean, std, paths_list
        
    def load_clean_from_folder(self, clean_folder, resize_size=160):
        self.clean_folder = clean_folder
        self.clean_images, self.orig_mean, self.orig_std, self.orig_paths = self._load_folder_for_facenet(
            clean_folder, resize_size=resize_size)
    
    def _undo_preprocess(self, images):
        restored_images = images.copy()
        restored_images  *= self.orig_std
        restored_images += self.orig_mean
        restored_images = np.clip(restored_images, 0.0, 255.0)
        return np.uint8(restored_images)
    
    def _compute_embeddings(self, model, images, layer_to_target):
        return model.predict(np.array(images), batch_size=len(images), layer_to_target=layer_to_target)
    
    def get_clean_for_display(self):
        return self._undo_preprocess(self.clean_images)
    
    def compute_clean_embeddings_with_model(self, model, layer_to_target):
        self.clean_embeddings = self._compute_embeddings(model, self.clean_images, layer_to_target)

In [5]:
class MyModel:
    def __init__(self, model_path, inputs=None, sess=None):
        if inputs is None:
            self.model_inputs = tf.placeholder(tf.float32, shape=(None, 160, 160, 3))
        else:
            self.model_inputs = inputs
        
        vars_before = tf.global_variables()
        self.net, self.endpoints = inference(
            self.model_inputs, keep_probability=1.0, bottleneck_layer_size=512, phase_train=False)
        vars_after = tf.global_variables()
        
        model_name = dataset_to_model_name["vggface2"]
        saver = tf.train.Saver(list(set(vars_after) - set(vars_before)))
        
        if sess is None:
            self.sess = tf.Session()
        else:
            self.sess = sess
            
        saver.restore(self.sess, model_path)
    
    def predict(self, inputs, batch_size, layer_to_target):
        return self.sess.run(self.endpoints[layer_to_target], feed_dict={self.model_inputs: inputs})
    
    def __del__(self):
        self.sess.close()

dataset_to_model_name = {
    "casia-webface": "20180408-102900",
    "vggface2": "20180402-114759"
}

dataset_to_ckpt_number = {
    "casia-webface": "90",
    "vggface2": "275"
}

def build_model(dataset_name, inputs=None, sess=None):
    model_name = dataset_to_model_name[dataset_name]
    model = MyModel(
        os.path.join(
            "/home/ivan/facenet/models",
            model_name,
            "model-{model_name}.ckpt-{ckpt_num}".format(
                model_name=dataset_to_model_name[dataset_name],
                ckpt_num=dataset_to_ckpt_number[dataset_name]
            )),
        inputs, 
        sess
    )
    return model

In [6]:
def path_for_id_clean(identity):
    if identity == "n000958":
        return "/data/vggface/test_perturbed_sampled/{id}/community_naive_mean/n000029/epsilon_0.0/png".format(id=identity)
    else:
        return "/data/vggface/test_perturbed_sampled/{id}/community_naive_mean/n000958/epsilon_0.0/png".format(id=identity)
    

In [7]:
def build_person(person_name, path_for_id_fn, model=None, layer_to_target=None):
    person_a = PersonImages(person_name)
    person_a.load_clean_from_folder(path_for_id_fn(person_a.person_name))
    if model is not None:
        print("Computing embeddings for", person_name)
        person_a.compute_clean_embeddings_with_model(model, layer_to_target)
    return person_a

In [8]:
vggface2_model = build_model("vggface2")
people = [
    build_person(person_name, path_for_id_clean, vggface2_model, layer_to_target) \
    for person_name in os.listdir("/data/vggface/test_perturbed_sampled")
]
del vggface2_model

INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from /home/ivan/facenet/models/20180402-114759/model-20180402-114759.ckpt-275
Computing embeddings for n001781
Computing embeddings for n009232
Computing embeddings for n000958
Computing embeddings for n003356
Computing embeddings for n008655
Computing embeddings for n008613
Computing embeddings for n004658
Computing embeddings for n001683
Computing embeddings for n002647
Computing embeddings for n009288
Computing embeddings for n005427
Computing embeddings for n002763
Computing embeddings for n002503
Computing embeddings for n003215
Computing embeddings for n005359
Computing embeddings for n005303
Computing embeddings for n007548
Computing embeddings for n000029
Computing embeddings for n009114


In [9]:
def build_vggface_model(inputs, sess):
    return build_model("vggface2", inputs, sess)

def build_casiawebface_model(inputs, sess):
    return build_model("casia-webface", inputs, sess)

In [10]:
def attack_batch_cw(
    build_model_fn, 
    input_images,
    target_vectors,
    learning_rate,
    epsilon,
    max_iters,
    layer_to_target
):
    input_images = np.array(input_images)
    batch_size, orig_h, orig_w, orig_c = input_images.shape
    print("input images mean {mean} and stddev {stddev}".format(mean=np.mean(input_images), stddev=np.std(input_images)))
    with tf.Graph().as_default():
        with tf.Session() as sess:
            targets_var = tf.get_variable(
                name="targets",
                shape=target_vectors.shape,
            )
            sess.run(tf.assign(targets_var, target_vectors))
            
            images_input_ph = tf.placeholder(
                tf.float32,
                name="input_images",
                shape=input_images.shape
            )
            
            noise_var = tf.get_variable(
                name="adversarial_noise",
                shape=input_images.shape,
                initializer=tf.initializers.truncated_normal(
                    mean=np.mean(input_images),
                    stddev=3 * np.std(input_images)
                )
            )
            sess.run(noise_var.initializer)
            
            images_plus_noise = images_input_ph + noise_var

            randomized_images_plus_noise = tf.image.random_brightness(
                images_plus_noise, 0.5)
        
            randomized_images_plus_noise = tf.image.random_crop(
                randomized_images_plus_noise, 
                [batch_size, orig_h - 10, orig_w - 10, 3]
            )

            randomized_images_plus_noise = tf.image.resize_images(
                randomized_images_plus_noise, [orig_h, orig_w])
            
            randomized_images_plus_noise += tf.random.normal(
                randomized_images_plus_noise.shape, 0.0, 0.75)
            
            randomized_images_plus_noise = tf.clip_by_value(
                randomized_images_plus_noise, input_images - epsilon, input_images + epsilon)
            
            model = build_model_fn(
                inputs=randomized_images_plus_noise, 
                sess=sess
            )
            
            model_outputs = model.endpoints[layer_to_target]
            assert np.array_equal(model_outputs.shape.as_list(), targets_var.shape.as_list()), "Shape mismatch"
            loss = tf.reduce_mean((model_outputs - targets_var) ** 2)
            
        
            total_loss = loss + 1e-6 * tf.nn.l2_loss(noise_var)
            
            vars_before = set(tf.global_variables())
            train_op = tf.train.AdamOptimizer(learning_rate).minimize(
                total_loss, var_list=[noise_var])
            vars_after = set(tf.global_variables())
            sess.run([v.initializer for v in list(vars_after - vars_before)])
            
            
            losses = []
            for i in range(max_iters):
                loss_value, total_loss_value, _ = sess.run(
                    [loss, total_loss, train_op], feed_dict={images_input_ph: input_images})
                assert not np.isnan(loss_value), "Loss_value is nan"
                losses.append(loss_value)
            
            final_imgs = sess.run(
                tf.clip_by_value(images_plus_noise, input_images - epsilon, input_images + epsilon),
                feed_dict={images_input_ph: input_images}
            )
            
            return final_imgs, losses

In [11]:
def generate_decoys_bigger_batches(
    people_list,
    attack_strategy,
    model_to_attack,
    learning_rate,
    epsilon,
    max_iters,
    layer_to_target
):
    if model_to_attack == "vggface2":
        model_build_fn = build_vggface_model
    elif model_to_attack == "casia-webface":
        model_build_fn = build_casiawebface_model
        
    for person in tqdm(people_list):
        num, height, width, channels = person.clean_images.shape
        person.adversarial_images = np.zeros((2*(len(people_list) - 1), height, width, channels))
        
        indx = 0
        
        images_to_make_adversarial = []
        targets_for_images = []
        
        for other_person in people_list:
            if person.person_name == other_person.person_name:
                continue
                
            current_chosen_indices = range(indx, indx + 2)
            images_to_make_adversarial.extend(np.take(
                person.clean_images, current_chosen_indices, axis=0))
            
            target_vector = np.mean(other_person.clean_embeddings, axis=0)
            targets_for_images.extend(np.array([
                target_vector for _ in range(len(current_chosen_indices))]))
            
            indx += 2
            
        images_to_make_adversarial = np.array(images_to_make_adversarial)
        
        all_adversarial_images, losses = attack_batch_cw(
            model_build_fn, 
            np.array(images_to_make_adversarial),
            np.array(targets_for_images),
            learning_rate,
            epsilon,
            max_iters,
            layer_to_target
        )
            
        fig, ax = plt.subplots()
        ax.plot(range(len(losses)), losses)
        plt.show()
        
        indx = 0
        for other_person in people_list:
            if person.person_name == other_person.person_name:
                continue
            
            save_dest = os.path.join(
                "/data/vggface/test_perturbed_sampled",
                person.person_name,
                "{attack_strategy}_{model}".format(attack_strategy=attack_strategy, model=model_to_attack),
                other_person.person_name
            )
            
            save_path = os.path.join(save_dest, "epsilon_{}".format(epsilon), "png")
            os.makedirs(save_path, exist_ok=True)
            existing_files = os.listdir(save_path)

            # Clean up folder if need be
            if len(existing_files) > 0:
                for f in existing_files:
                    os.remove(os.path.join(save_path, f))
            
            for i in range(indx, indx + 2):
                orig_name = person.orig_paths[i].split("/")[-1]
                person.adversarial_images[i] = all_adversarial_images[i]
                full_save_path = os.path.join(save_path, orig_name)
                print("Saving to", save_path)
                Image.fromarray(
                    person._undo_preprocess(person.adversarial_images[i])
                ).save(full_save_path)

            indx += 2

In [ ]:
for epsilon in [0.1, 0.25, 0.5]:
    generate_decoys_bigger_batches(
        people,
        "mean_{}".format(layer_to_target),
        "vggface2",
        0.01,
        epsilon,
        2000,
        layer_to_target
    )

  0%|          | 0/19 [00:00<?, ?it/s]

input images mean 9.97825910786787e-09 and stddev 1.0000001192092896
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from /home/ivan/facenet/models/20180402-114759/model-20180402-114759.ckpt-275


In [ ]:
for epsilon in [0.1, 0.25, 0.5]:
    generate_decoys_bigger_batches(
        people,
        "mean_{}".format(layer_to_target),
        "casia-webface",
        0.01,
        epsilon,
        2000,
        layer_to_target
    )

In [ ]:
class FacialVectorsDatabase:
    def __init__(self, folder, model_name, epsilon, attack_strategy, image_format, num_clean, include_decoys=True):
        self.associated_identities = []
        self.vectors = []
        self.associated_paths = []
        
        identities = os.listdir(folder)
        
        
        self.graph = tf.Graph()
        
        with self.graph.as_default():
            self.model = build_model(model_name)

            for person_name in tqdm(identities):
                attack_subfolder = os.path.join(folder, person_name, attack_strategy)

                protected = os.listdir(attack_subfolder)
                
                if include_decoys:
                    for indx, other_identity in enumerate(protected):
                        protected_folder = os.path.join(
                            attack_subfolder, 
                            other_identity, 
                            "epsilon_{eps}".format(eps=epsilon), 
                            image_format
                        )

                        self._add_folder_for_person(
                            protected_folder, 
                            person_name,
                            exclude_endings=None,
                            max_imgs=-1
                        )
                        
                # These were the images we used for this person for adversarial modification
                used_images = [
                    x.split("/")[-1] \
                    for indx, x in enumerate(self.associated_paths) \
                    if self.associated_identities[indx] == person_name
                ]
                
                clean_folder = os.path.join(
                    folder, person_name, "community_naive_mean", protected[0], "epsilon_0.0", "png")
                self._add_folder_for_person(
                    clean_folder, 
                    person_name,
                    exclude_endings=set(used_images),
                    max_imgs=num_clean
                )
        
                
    def _load_one_facenet(self, path, crop_box=None, resize_size=None, prewhiten_img=True):
        img = Image.open(path).convert("RGB")
        
        if crop_box:
            img = img.crop(crop_box)
            
        if resize_size:
            img = img.resize((resize_size, resize_size))
            
        img = (np.array(img)).astype(np.float32)
        
        if prewhiten_img:
            img = prewhiten(img)
            
        return img
        

    def _load_folder_for_facenet(self, folder, exclude_endings=None, max_imgs=-1):
        paths_list = glob.glob(os.path.join(folder, "*"))
        len_before = len(paths_list)
        if not (exclude_endings is None):
            paths_list = [x for x in paths_list if not (x.split("/")[-1] in exclude_endings)]
            
        if max_imgs > 0:
            paths_list = paths_list[:max_imgs]
        
        final_imgs = [
            self._load_one_facenet(
                img_path, 
                prewhiten_img=False, 
                resize_size=None, 
                crop_box=None) for img_path in paths_list
        ]

        final_imgs = np.array(final_imgs)
        final_imgs = prewhiten(np.array(final_imgs))
        return final_imgs, paths_list

    
    def _compute_embeddings(self, images):
        return self.model.predict(np.array(images), batch_size=len(images))
    
    
    def _add_folder_for_person(self, folder, person_name, exclude_endings=None, max_imgs=-1):
        images, paths_list = self._load_folder_for_facenet(folder, exclude_endings, max_imgs=max_imgs)
        
        if images is None:
            return
        
        vectors = self._compute_embeddings(images)
        self.vectors.extend(vectors)
        self.associated_identities.extend([person_name for _ in range(len(vectors))])
        self.associated_paths.extend(paths_list)
    
    def nearest_neighbor_to_img_at_path(self, query_path):
        with self.graph.as_default():
            query_vector = self._compute_embeddings(
                np.expand_dims(self._load_one_facenet(query_path, resize_size=160), axis=0))
        distances = pairwise_distances(query_vector, self.vectors, metric="cosine")[0]
        min_dist_indx = np.argmin(distances)
        return self.associated_identities[min_dist_indx], self.associated_paths[min_dist_indx]
    

def full_path_to_info(path):
    split = path.split("/")
    if path.startswith("/data/vggface/test_perturbed_sampled"):
        # split of path breaks down like this:
        #     0 
        #     1 data
        #     2 vggface
        #     3 test_perturbed_sampled
        #     4 {protector}
        #     5 community_naive_mean
        #     6 {protected}
        #     7 epsilon_0.0
        #     8 png
        #     9 35.png
        return "-".join([split[4], split[6], split[7], split[9]])
    else:
        # split of path breaks down like this:
        #     0 
        #     1 data
        #     2 vggface
        #     3 test_query_antisampled
        #     4 {protector}
        #     5 image_name.jpeg
        return "-".join([split[4], split[5]])
    
    
def measure_local_recall(
    faces_database,
    image_directory="/data/vggface/test_query_antisampled",
    num_query=10,
    verbose=False
):
    discovery = []
    true = []
    identified_as = []
    
    paths_of_query = []
    paths_of_nearest = []

    for protector in os.listdir(image_directory):
        # We are sourcing query photos from epsilon_0.0.
        # In those cases, all subfolders in the "protected" identity have the same, clean
        # photo of the protector, so we just pick any single one that exists (e.g. n000958)
        # For the case where n000958 is itself the protector, n000958 is not present in its protected
        # subfolders, so we pick n000029 without loss of generality.
        if protector == "n000958":
            protected = "n000029"
        else:
            protected = "n000958"

        query_photos_paths = sorted(glob.glob(
            os.path.join(image_directory, protector, "*")
        ))

        
        for i in np.random.choice(len(query_photos_paths), num_query):
            chosen_path = query_photos_paths[i]
            top_identity, top_identity_path = faces_database.nearest_neighbor_to_img_at_path(chosen_path)
            
            paths_of_query.append(full_path_to_info(chosen_path))
            paths_of_nearest.append(full_path_to_info(top_identity_path))
            
            true.append(protector)
            identified_as.append(top_identity)

            if top_identity == protector:
                discovery.append(1.0)
            else:
                discovery.append(0.0)

    if verbose:
        for true_id, recognized_id, query, nearest in zip(true, identified_as, paths_of_query, paths_of_nearest):
            print("Face of {true_id} identitifed as {recognized_id}. Nearest neighbor to {query} was {nearest}".format(
                true_id=true_id, recognized_id=recognized_id, query=query, nearest=nearest))
        
    return sum(discovery)/len(discovery)

In [ ]:
def get_recall_once(
    network_to_evaluate,
    epsilon,
    attack_name,
    num_clean,
    include_adversarial,
    run
):
    faces_db = FacialVectorsDatabase(
        folder="/data/vggface/test_perturbed_sampled",
        model_name=network_to_evaluate,
        epsilon=epsilon,
        attack_strategy=attack_name,
        image_format="png", 
        num_clean=num_clean,
        include_decoys=include_adversarial
    )
    recall_at_k1 = measure_local_recall(faces_db, num_query=10, verbose=False)

    faces_db.model.sess.close()
    
    return {
        "num_clean": num_clean, 
        "run": run,
        "recall": recall_at_k1,
        "epsilon": epsilon
    }

def generate_results_plot(
    network_to_evaluate,
    attack_name
):
    results = []
    
    # get a run with all clean ones and no decous
    res = get_recall_once(
        network_to_evaluate,
        0.5, # epsilon is irrelevant
        attack_name,
        -1, # negative or 0 number includes all
        False,
        11
    )
    results.append(res)
    
    for epsilon in [0.1, 0.25, 0.5]:
        for num_clean in range(1, 10, 2):
            for run in range(10):
                res = get_recall_once(
                    network_to_evaluate,
                    epsilon,
                    attack_name,
                    num_clean,
                    True,
                    run
                )
                results.append(res)
                
    results_pd = pd.DataFrame(results)

    results_pd["decoys_vs_clean"] = 36 / results_pd["num_clean"]
    results_pd[results_pd.decoys_vs_clean < 0.0]["decoys_vs_clean"] = 0.0
    
    results_pd.to_csv(
        "/home/ivan/pascal_adversarial_faces/results/recall_at_1_{attack_name}_{network_to_evaluate}.csv".format(
            network_to_evaluate=network_to_evaluate,
            attack_name=attack_name
        ))

    ax = sns.lineplot(
        data=results_pd, x="decoys_vs_clean", y="recall", style="epsilon", markers=True)
    ax.set_title("Attack {attack} evaluated on {network_to_evaluate}")

In [ ]:
generate_results_plot(
    "casia-webface",
    "mean_{}_vggface2".format(layer_to_target)
)

In [ ]:
generate_results_plot(
    "vggface2",
    "mean_{}_casia-webface".format(layer_to_target)
)

In [ ]:
generate_results_plot(
    "vggface2",
    "mean_{}_vggface2".format(layer_to_target)
)

In [ ]:
generate_results_plot(
    "casia-webface",
    "mean_{}_casia-webface".format(layer_to_target)
)